# Implementacion del modelo mátematico planteado en la etapa 1:

In [14]:
import math
import pyomo.environ as pyo

DEPOT = 0
CLIENTES = [1, 2, 3]
NODOS = [DEPOT] + CLIENTES

# matriz de distancias 
w = {
    (0, 0): 0, (0, 1): 8, (0, 2): 7, (0, 3): 9,
    (1, 0): 8, (1, 1): 0, (1, 2): 5, (1, 3): 6,
    (2, 0): 7, (2, 1): 5, (2, 2): 0, (2, 3): 9,
    (3, 0): 9, (3, 1): 6, (3, 2): 9, (3, 3): 0
}

# Parámetros
demand = {1: 70, 2: 60, 3: 80}  # Total: 210 kg
V = [1, 2]  
rv = {1: 150, 2: 100}      # capacidades (kg)
tv = {1: 50, 2: 40}        # autonomías (km)
ev = {1: 10.0, 2: 12.0}    # rendimiento (km/L)

# Costos 
ct = 3000      # costo transporte por km
cm = 500       # costo mantenimiento por km
co = 50000     # costo operativo por vehículo
pf = 12000     # precio del combustible por litro

# funciones auxiliares
def safe_value(var):
    try:
        return pyo.value(var)
    except:
        return None

def fmt_num(x, fmt="{:,.0f}"):
    return fmt.format(x) if x is not None and x != "" else "N/A"


# model
model = pyo.ConcreteModel()

# Conjuntos
model.N = pyo.Set(initialize=NODOS)
model.C = pyo.Set(initialize=CLIENTES)
model.V = pyo.Set(initialize=V)
model.D = pyo.Set(initialize=[DEPOT])

# Parámetros
model.w = pyo.Param(model.N, model.N, initialize=w, within=pyo.NonNegativeReals)
model.q = pyo.Param(model.C, initialize=demand, within=pyo.NonNegativeReals)
model.r = pyo.Param(model.V, initialize=rv, within=pyo.NonNegativeReals)
model.tau = pyo.Param(model.V, initialize=tv, within=pyo.NonNegativeReals)
model.e = pyo.Param(model.V, initialize=ev, within=pyo.PositiveReals)

model.ct = pyo.Param(initialize=ct)
model.cm = pyo.Param(initialize=cm)
model.co = pyo.Param(initialize=co)
model.pf = pyo.Param(initialize=pf)

# costo por arco 
def s_init(model, i, j, v):
    return (pyo.value(model.ct) + pyo.value(model.cm) + pyo.value(model.pf) / pyo.value(model.e[v])) * pyo.value(model.w[i, j])
model.s = pyo.Param(model.N, model.N, model.V, initialize=s_init, within=pyo.NonNegativeReals)

# Variables 
model.x = pyo.Var(model.N, model.N, model.V, domain=pyo.Binary)
model.y = pyo.Var(model.V, domain=pyo.Binary)
model.u = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals, bounds=(0, max(rv.values())))

def no_self_arcs_rule(m, i, v):
    return m.x[i, i, v] == 0
model.no_self_arcs = pyo.Constraint(model.N, model.V, rule=no_self_arcs_rule)

# Función Objetivo
def obj_rule(m):
    return sum(m.s[i, j, v] * m.x[i, j, v] for i in m.N for j in m.N for v in m.V) + sum(m.co * m.y[v] for v in m.V)
model.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Restricciones
def flow_cons_rule(m, v, c):
    return sum(m.x[i, c, v] for i in m.N if i != c) - sum(m.x[c, j, v] for j in m.N if j != c) == 0
model.flow_cons = pyo.Constraint(model.V, model.C, rule=flow_cons_rule)

def autonomy_rule(m, v):
    return sum(m.x[i, j, v] * m.w[i, j] for i in m.N for j in m.N if i != j) <= m.tau[v]
model.autonomy = pyo.Constraint(model.V, rule=autonomy_rule)

def mtz_rule(m, c, j, v):
    if c == j:
        return pyo.Constraint.Skip
    return m.u[c, v] - m.u[j, v] + m.r[v] * m.x[c, j, v] <= m.r[v] - m.q[j]
model.mtz = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

def u_lower_rule(m, c, v):
    return m.q[c] <= m.u[c, v]
model.u_lower = pyo.Constraint(model.C, model.V, rule=u_lower_rule)

def u_upper_rule(m, c, v):
    return m.u[c, v] <= m.r[v]
model.u_upper = pyo.Constraint(model.C, model.V, rule=u_upper_rule)

def depart_from_depot_rule(m, v):
    return sum(m.x[DEPOT, j, v] for j in m.C) == m.y[v]
model.depart_depot = pyo.Constraint(model.V, rule=depart_from_depot_rule)

def return_to_depot_rule(m, v):
    return sum(m.x[i, DEPOT, v] for i in m.C) == m.y[v]
model.return_depot = pyo.Constraint(model.V, rule=return_to_depot_rule)

def cover_once_rule(m, j):
    return sum(m.x[i, j, v] for v in m.V for i in m.N if i != j) == 1
model.cover_once = pyo.Constraint(model.C, rule=cover_once_rule)

result = solver.solve(model, tee=False)

# Calcular costos
costo_variable = 0
costo_fijo = 0
rutas_vehiculos = {}

for v in model.V:
    y_val = pyo.value(model.y[v])
    if y_val > 0.5:  # Vehículo utilizado
        costo_fijo += co
        
        # Reconstruir ruta
        route = ["CD0"]
        current = DEPOT
        visited = set()
        
        for _ in range(len(NODOS) + 2):
            found_next = False
            for j in NODOS:
                if j != current:
                    x_val = pyo.value(model.x[current, j, v])
                    if x_val is not None and x_val > 0.5:
                        if j == DEPOT:
                            route.append("CD0")
                            current = j
                            found_next = True
                            break
                        else:
                            route.append(f"C{j}")
                            visited.add(j)
                            current = j
                            found_next = True
                            break
            if not found_next or current == DEPOT:
                break
        
        # Calcular métricas de la ruta
        distancia_ruta = 0
        carga_ruta = 0
        
        for i in range(len(route) - 1):
            node_i = DEPOT if route[i] == "CD0" else int(route[i][1:])
            node_j = DEPOT if route[i + 1] == "CD0" else int(route[i + 1][1:])
            distancia_ruta += w[node_i, node_j]
            
            # Sumar costo variable para este arco
            s_val = (ct + cm + pf/ev[v]) * w[node_i, node_j]
            costo_variable += s_val
        
        for client in visited:
            carga_ruta += demand[client]
        
        rutas_vehiculos[v] = {
            'ruta': route,
            'distancia': distancia_ruta,
            'carga': carga_ruta,
            'clientes': visited
        }

# mostrasr rtsults
costo_total = costo_variable + costo_fijo
print("Resumen")
print(f"Costo variable: {costo_variable:,.0f} COP")
print(f"Costo fijo: {costo_fijo:,.0f} COP")
print(f"Costo total: {costo_total:,.0f} COP")

# Mostrar rutas optimizadas
print("Rutas")
for v, datos in rutas_vehiculos.items():
    print(f"\nVehículo {v} (Capacidad: {rv[v]}kg, Autonomia: {tv[v]}km):")
    print(f"  Ruta: {' → '.join(datos['ruta'])}")
    print(f"  Distancia: {datos['distancia']:.1f} km / {tv[v]} km")
    print(f"  Carga: {datos['carga']} kg / {rv[v]} kg")
    print(f"  Clientes: {sorted([f'C{c}' for c in datos['clientes']])}")

# Validación 
print(f"Total demanda: {sum(demand.values())} kg")


print("Resulktados")

for v, datos in rutas_vehiculos.items():
    print(f"\nVehículo {v}:")
    print(f"Ruta completa: {' → '.join(datos['ruta'])}")
    
    #  distancia tramo por tramo
    distancia_detallada = 0
    costo_vehiculo = 0
    for i in range(len(datos['ruta']) - 1):
        node_i = DEPOT if datos['ruta'][i] == "CD0" else int(datos['ruta'][i][1:])
        node_j = DEPOT if datos['ruta'][i + 1] == "CD0" else int(datos['ruta'][i + 1][1:])
        tramo_distancia = w[node_i, node_j]
        tramo_costo = (ct + cm + pf/ev[v]) * tramo_distancia
        
        distancia_detallada += tramo_distancia
        costo_vehiculo += tramo_costo
        
        print(f"  {datos['ruta'][i]} → {datos['ruta'][i+1]}: {tramo_distancia} km × {ct + cm + pf/ev[v]:.0f} COP/km = {tramo_costo:,.0f} COP")
    
    print(f"Distancia total: {distancia_detallada} km")
    print(f"Costo variable vehículo {v}: {costo_vehiculo:,.0f} COP")

Resumen
Costo variable: 171,100 COP
Costo fijo: 100,000 COP
Costo total: 271,100 COP
Rutas

Vehículo 1 (Capacidad: 150kg, Autonomia: 50km):
  Ruta: CD0 → C3 → C1 → CD0
  Distancia: 23.0 km / 50 km
  Carga: 150 kg / 150 kg
  Clientes: ['C1', 'C3']

Vehículo 2 (Capacidad: 100kg, Autonomia: 40km):
  Ruta: CD0 → C2 → CD0
  Distancia: 14.0 km / 40 km
  Carga: 60 kg / 100 kg
  Clientes: ['C2']
Total demanda: 210 kg
Resulktados

Vehículo 1:
Ruta completa: CD0 → C3 → C1 → CD0
  CD0 → C3: 9 km × 4700 COP/km = 42,300 COP
  C3 → C1: 6 km × 4700 COP/km = 28,200 COP
  C1 → CD0: 8 km × 4700 COP/km = 37,600 COP
Distancia total: 23 km
Costo variable vehículo 1: 108,100 COP

Vehículo 2:
Ruta completa: CD0 → C2 → CD0
  CD0 → C2: 7 km × 4500 COP/km = 31,500 COP
  C2 → CD0: 7 km × 4500 COP/km = 31,500 COP
Distancia total: 14 km
Costo variable vehículo 2: 63,000 COP
